In [70]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import urllib.request as ur

from time import sleep
from random import randint
import re

import requests
from bs4 import BeautifulSoup

In [71]:
headers = {"Accept-Language": "en-US, en;q=0.5"}
direction = ["North","East","West","South"]
pages = np.arange(0,161)

In [502]:
location = []
beds = []
price = []
baths = []
location_sector = []
parking = []
laundry = []    
furnished = []

In [505]:
counter = 0
try:
    for page in pages:
        counter = counter+1
        page = requests.get("https://offcampus.osu.edu/search-housing.aspx?page=" + str(page) + "&pricefrom=0&sort=1", headers = headers,timeout =10)
        url = "https://offcampus.osu.edu/search-housing.aspx?page=" + str(page) + "&pricefrom=0&sort=1"
        soup = BeautifulSoup(page.text, "html.parser")
        appartment_div = soup.find_all('div', class_='o-row__col o-row__col--6of12@md o-row__col--4of12@xl')
        sleep(randint(2,10))

        for container in appartment_div:
            # location
            name = container.h2.a.text
            match = next((x for x in direction if x in name), "Null")
            location.append(match)
    
            info = container.find_all("dd")       
    
            ##bed
            bed = int(info[1].text[0])
            beds.append(bed)
    
            ##Baths
            bath = int(info[2].text[0])
            baths.append(bath)
            
        
          
        
            possible_links = soup.find_all(lambda tag: tag.name == 'div' and 
                            tag.get('class') == ['c-propertycard__info'])
        
            listings = []
        
  
            
            for div in possible_links:
                listings.append(div.a['href'])
            
            for list_e in listings:
                page = requests.get("https://offcampus.osu.edu/" + str(list_e), headers = headers, timeout =10)
                soup = BeautifulSoup(page.text, "html.parser")
                ul = soup.select('div[class="o-row__col o-row__col--6of12@lg"] li', recursive=True)
                lis_e = []
                for li in ul:
                    lis = []
                    lis.append(li.contents)
                    lis_e.extend(lis)
        
                    
                pattern = re.compile('(?<=University District:)\s(\w+)')
                matches = pattern.search(str(lis_e))
                if matches:
                    location_sector.append(matches.group(1))
                else:
                    location_sector.append("None")
                pattern = re.compile('(?<=Off-street Parking:).*\s.*(Yes|No)')
                matches = pattern.search(str(lis_e))
                if matches:
                    parking.append(matches.group(1))
                else:
                    parking.append("None")
                pattern = re.compile('Laundry facilities in the (unit|building)')
                matches = pattern.search(str(lis_e))
                if matches:
                    laundry.append(matches.group(1))
                else:
                    laundry.append("None")
                pattern = re.compile('(?<=Furnished).*(Yes|No)')
                matches = pattern.search(str(lis_e))
                if matches:
                    furnished.append(matches.group(1))
                else:
                    furnished.append("None")
    
                c = lis_e[1][1].split()
                if len(c) ==  1:
                    price.append(float(c[0].replace('$','').replace(",",'')))
                else:
                    num_1 = float(c[2].replace('$','').replace(",",''))
                    price.append((num_1))
                
except requests.exceptions.Timeout:
            print(url)
            print(counter)
            print("Timeout occurred")
    

In [580]:
df = pd.DataFrame(columns = ['location','beds','price','baths','location_sector','parking','laundry','furnished'])

In [581]:
df['location'] = location
df['beds'] = beds
df['price'] = price
df['baths'] = baths
df['location_sector'] = location_sector
df['parking'] = parking
df['laundry'] = laundry
df['furnished'] = furnished

In [583]:
df['price'] = np.where((df['price']<=1000) & (df['beds']>2), df['price']*df['beds'], df['price'])

In [586]:
df.to_csv('Listing.csv')